<center>

# SSO Assignment 3

#### **Authors:** Group 13 
#### Abhinav Agnihotry, Aryen Masoudpay, Shantanu Motiani  

 **Date:** 19th October, 2024

</center>

## Exercise 3.1

### Part (a)

We need to create an integer linear optimization model that minimizes the total tardiness cost, which is incurred for each hour a job exceeds its due date.

Decision variables:
1. start[j]: Start time of job j.
2. tardiness[j]: The tardiness of job j (the time by which it exceeds its due date).

Objective: Minimize the sum of tardiness for all jobs.

Constraints:
1. Jobs can only start after their release time.
2. Jobs should have sufficient time to finish before the next job starts.
3. Tardiness should be calculated based on the difference between finish time and due date.

In [10]:
import pulp as lp

jobs = list(range(1, 11))
duration = [4, 5, 3, 5, 7, 1, 0, 3, 2, 10]
release_time = [3, 4, 7, 11, 10, 0, 0, 10, 0, 15]
due_date = [11, 12, 20, 25, 20, 10, 30, 30, 10, 20]

# Create a problem instance
prob = lp.LpProblem("Single-Machine_Scheduling", lp.LpMinimize)

# Decision variables: start times and tardiness for each job
start = lp.LpVariable.dicts("Start", jobs, lowBound=0, cat="Continuous")
tardiness = lp.LpVariable.dicts("Tardiness", jobs, lowBound=0, cat="Continuous")

# Objective: Minimize total tardiness
prob += lp.lpSum(tardiness[j] for j in jobs), "Total_Tardiness"

# Constraints
for j in jobs:
    # Job can only start after its release time
    prob += start[j] >= release_time[j-1], f"Release_time_constraint_{j}"
    
    # Tardiness constraint (if finish time exceeds due date)
    prob += tardiness[j] >= (start[j] + duration[j-1] - due_date[j-1]), f"Tardiness_constraint_{j}"
    
    # No overlap between jobs
    if j < 10:  # For jobs from 1 to 9
        prob += start[j] + duration[j-1] <= start[j+1], f"No_overlap_constraint_{j}"

# Solve the problem
prob.solve()

# Output the results
print("Optimal Schedule:")
for j in jobs:
    print(f"Job {j}: Start time = {start[j].varValue}, Tardiness = {tardiness[j].varValue}")

# Report on total tardiness
total_tardiness = sum([tardiness[j].varValue for j in jobs])
print(f"\nTotal Cost of Tardiness: {total_tardiness} euros")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/simulation/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/qw/kndrzd253bbb5qrt4cz3wxqr0000gn/T/ac7d39d1a869494facfc23992dabfa58-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/qw/kndrzd253bbb5qrt4cz3wxqr0000gn/T/ac7d39d1a869494facfc23992dabfa58-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 34 COLUMNS
At line 93 RHS
At line 123 BOUNDS
At line 124 ENDATA
Problem MODEL has 29 rows, 20 columns and 48 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-29) rows, 0 (-20) columns and 0 (-48) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 72
After Postsolve, objective 72, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 72 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions chang

### Part (b)

Now we introduce two sieve types. Jobs 1, 3, 4, 6, and 10 require sieve type 1, and the others require sieve type 2. Changing the sieve adds 1 hour, so we need to modify the model to account for this extra time whenever a switch between jobs using different sieve types occurs.

New constraint
1. Add 1 hour between jobs if they require different sieves.

In [6]:
# Sieve type for each job (1 for type 1, 2 for type 2)
sieve_type = {1: 1, 2: 2, 3: 1, 4: 1, 5: 2, 6: 1, 7: 2, 8: 2, 9: 2, 10: 1}

# Create a problem instance
prob = lp.LpProblem("Single-Machine_Scheduling_with_Sieve_Change", lp.LpMinimize)

# Decision variables: start times and tardiness for each job
start = lp.LpVariable.dicts("Start", jobs, lowBound=0, cat="Continuous")
tardiness = lp.LpVariable.dicts("Tardiness", jobs, lowBound=0, cat="Continuous")

# Objective: Minimize total tardiness
prob += lp.lpSum(tardiness[j] for j in jobs), "Total_Tardiness"

# Constraints
for j in jobs:
    # Job can only start after its release time
    prob += start[j] >= release_time[j-1], f"Release_time_constraint_{j}"
    
    # Tardiness constraint (if finish time exceeds due date)
    prob += tardiness[j] >= (start[j] + duration[j-1] - due_date[j-1]), f"Tardiness_constraint_{j}"
    
    # No overlap between jobs
    if j < 10:  # For jobs from 1 to 9
        if sieve_type[j] == sieve_type[j+1]:
            # No sieve change, normal no overlap constraint
            prob += start[j] + duration[j-1] <= start[j+1], f"No_overlap_constraint_{j}_{j+1}"
        else:
            # Sieve change required, add 1 hour
            prob += start[j] + duration[j-1] + 1 <= start[j+1], f"Sieve_change_constraint_{j}_{j+1}"

# Solve the modified problem
prob.solve()

# Output the new results
print("Optimal Schedule with Sieve Change:")
for j in jobs:
    print(f"Job {j}: Start time = {start[j].varValue}, Tardiness = {tardiness[j].varValue}")

# Report on total tardiness
total_tardiness = sum([tardiness[j].varValue for j in jobs])
print(f"\nTotal Cost of Tardiness with Sieve Change: {total_tardiness} euros")

Optimal Schedule with Sieve Change:
Job 1: Start time = 3.0, Tardiness = 0.0
Job 2: Start time = 8.0, Tardiness = 1.0
Job 3: Start time = 14.0, Tardiness = 0.0
Job 4: Start time = 17.0, Tardiness = 0.0
Job 5: Start time = 23.0, Tardiness = 10.0
Job 6: Start time = 31.0, Tardiness = 22.0
Job 7: Start time = 33.0, Tardiness = 3.0
Job 8: Start time = 33.0, Tardiness = 26.0
Job 9: Start time = 36.0, Tardiness = 8.0
Job 10: Start time = 39.0, Tardiness = 29.0

Total Cost of Tardiness with Sieve Change: 99.0 euros


## Exercise 3.2  
**Project Planning**

### Part (a)

The project was simulated 10,000 times to approximate the expected finish time. Activity durations were drawn from an exponential distribution using the inverse transformation method, where the rate parameter \(\lambda\) is the reciprocal of the expected duration. The sampling formula used in Excel was:

$$
x = -\frac{\ln(\text{RAND()})}{\lambda}
$$

Each activity's finish time was computed based on its dependencies. Activities 2, 3, and 4 started after Activity 1, while Activities 5 and 6 began after Activities 2 & 3, and 3 & 4, respectively. Activity 7, the final activity, started after Activities 5 and 6, with its finish time determining the overall project duration.

After 10,000 simulations, the expected project finish time was found to be **14.007 days**.


<center>
<div>
<img src="./ex_3.2a.png" width="700"/>
</div>
</center>

### Part (b)

To determine the 95% confidence interval, the following formula was employed:

$$
\bar{x} - t_{\alpha} \cdot \frac{s}{\sqrt{n}} \quad \text{and} \quad \bar{x} + t_{\alpha} \cdot \frac{s}{\sqrt{n}}
$$

where $\bar{x}$ represents the sample mean, $t_{\alpha/2} = 1.96$ for $1000 - 1$ degrees of freedom, $s$ denotes the sample standard deviation, and $n$ indicates the number of simulations. The calculated values for the 95% confidence interval for the expected project completion time are presented in the figure below:


<center>
<div>
<img src="./ex_3.2b.png" width="300"/>
</div>
</center>

### Part (c)

To estimate the 95% confidence interval for the likelihood of the project exceeding 12 days, a new column was introduced. This column assigns a value of 1 for simulations where the project duration surpasses 12 days and a value of 0 for all other cases. The sample probability was then determined by calculating the proportion of simulations that resulted in completion times greater than 12 days, yielding a figure of 56.61%. By applying this probability and the following formula, the confidence interval was computed:

$$
\hat{p} - z_{\alpha/2} \cdot \sqrt{\frac{\hat{p}(1 - \hat{p})}{n}}, \quad \hat{p} + z_{\alpha/2} \cdot \sqrt{\frac{\hat{p}(1 - \hat{p})}{n}}
$$

The findings suggest that we can be 95% confident that the true probability of the project taking more than 12 days lies within the range of 55.79% to 57.42%.


<center>
<div>
<img src="./ex_3.2c.png" width="300"/>
</div>
</center>

## Exercise 3.3
**Optimal Hotel Prizes**

### Part (a)

The demand for nine different price points was simulated 100,000 times using a binomial distribution. For each price, the expected average revenue and its standard deviation were then calculated. As shown in the figure below, the price of €150.00 yields the highest expected revenue, with a mean of €1,168.77 and a standard deviation of €277.24. Among the price points, this one maximizes revenue, outperforming the others in terms of expected earnings.


<center>
<div>
<img src="./ex_3.3a1.png" width="700"/>
</div>
</center>

To confirm whether €150 is indeed the optimal price, we must calculate the 95% confidence intervals (CIs) for the differences between the expected mean revenues of €150 and the other price points. If these confidence intervals do not include zero, it would provide sufficient evidence that €150 outperforms the other prices. The differences will be tested using the following adjusted significance level for multiple comparisons:

$$
\alpha^* = 1 - \frac{\sqrt{|S| - 1}}{\sqrt{1 - \alpha}} = 1 - \frac{\sqrt{9 - 1}}{\sqrt{0.95}} = 1 - \frac{8}{\sqrt{0.95}} \approx 0.006
$$

This adjusted significance level, $\alpha^*$, of approximately 0.006 corresponds to an inverse of 2.49 (beta), which will be used as the t-score for calculating the confidence intervals. The formula for determining the confidence intervals for the differences in expected means is as follows:

$$
CI = \left( \bar{X}_1 - \bar{X}_2 \right) \pm t_{\beta} \cdot SE
$$

This format is fully compatible with Jupyter notebooks for displaying the text, equations, and structure.

<center>
<div>
<img src="./ex_3.3a2.png" width="500"/>
</div>
</center>

With 95% confidence, €150 is the optimal price, as none of the confidence intervals include zero.

### Part (b)


The simulation budget has now been set to a total of 10,000 simulations, with only 900 simulations conducted in step 1. As in part a), the 9 price points were simulated, but this time, each was simulated 100 times (since $ \frac{900}{9} = 100 ) $. The mean expected revenue and standard deviations were calculated, as shown in the figure. Pairwise t-tests were then performed to compare all prices at a significance level of 0.05. 
<center>
<div>
<img src="./ex_3.3b1.png" width="500"/>
</div>
</center>

It’s important to note that the Sidak correction was applied to adjust for multiple comparisons. The adjusted alpha (\($\alpha^*$) is calculated as follows:

$$
\alpha^* = 1 - \frac{\sqrt{|S| - 1}}{\sqrt{1 - \alpha}} = 1 - \frac{\sqrt{9 - 1}}{\sqrt{0.95}} = 1 - \frac{8}{\sqrt{0.95}} \approx 0.006
$$

This $\alpha^*$ of 0.006 corresponds to an inverse of 2.54 (beta). The pairwise t-tests were conducted to identify and discard inferior price solutions.

<center>
<div>
<img src="./ex_3.3b2.png" width="500"/>
</div>
</center>



As shown in the figure above, five prices—€100, €110, €120, €170, and €180—can be eliminated from the set of candidate solutions. These prices were discarded because the null hypothesis, "price $ p_i $ is not worse than price $ p_i' $," could be rejected.

The remaining four prices—€130, €140, €150, and €160—can now be simulated with the remaining budget of $ 10,000 - 900 = 9,100 $ simulations. This results in 2,275 simulations per price $ \left( \frac{9,100}{4} = 2,275 \right) $. The figure below illustrates the simulation of these four prices, along with their calculated mean revenue and standard deviation.

<center>
<div>
<img src="./ex_3.3b3.png" width="800"/>
</div>
</center>

